#Belajar PySpark - Dataframe when dan otherwise

Pada artikel sebelumnya sudah dibahas tentang filter pyspark DataFrame dengan fungsi withColumn, maka kali ini kita akan belajar mengenai penggunaan fungsi when dan otherwise.

Fungsi when dan otherwise digunakan untuk memilih sebuah nilai dengan melakukan pengecekan beberapa kondisi secara bertahap. Fungsi ini serupa dengan ekspresi CASE WHEN dalam perintah SQL. Penerapannya yang paling sering adalah untuk menambahkan kolom baru berdasarkan nilai kolom-kolom lain yang sudah ada.

Sintaks rangkaian when-otherwise adalah sebagai berikut :

`F.when(kondisi_1, output_1)`
<br>&emsp;&emsp;&emsp;`.when(kondisi_2, output_2)`
<br>&emsp;&emsp;&emsp;`…`
<br>&emsp;&emsp;&emsp;`.when(kondisi_N, output_N)`
<br>&emsp;&emsp;&emsp;`.otherwise(output_default))`

Perhatikan bahwa dalam rangkain kode di atas kita memanggil 2 jenis fungsi `when()` yang berbeda.

1. Fungsi when pertama adalah fungsi dari package `pyspark.sql.functions` Pada contoh di atas kita memanggilnya dengan `F.when()` karena kita melakukan import `pyspark.sql.functions` sebagai F dengan perintah `pyspark.sql.functions as F`
2. Fungsi when kedua dan seterusnya adalah fungsi dari object `pyspark.sql.Column`.

Meskipun demikian, kedua fungsi tersebut melakukan hal yang sama, yaitu menerima parameter berupa kondisi, dan mengembalikan object bertipe Column.

Pada artikel ini kita akan membahas 3 cara menambahkan kolom baru di Spark DataFrame berdasar kolom yang sudah ada, yaitu :

1. Menggunakan withColumn dan when-otherwise
2. Menggunakan select dan when-otherwise
2. Menggunakan select dan when-otherwise untuk menambahkan beberapa kolom sekaligus

Pertama-tama kita install package pyspark.


In [ ]:
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=658dd9cc9f76106b7f2190812d188752f276d0b00cd6bcac67347d15426a3867
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Berikut adalah package yang akan kita gunakan dalam artikel ini:

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Selanjutnya kita buat object SparkSession dengan nama aplikasi *‘Belajar Pyspark - When Otherwise’*.


In [ ]:
spark = SparkSession.builder.appName('Belajar PySpark - When Otherwise').getOrCreate()

DataFrame yang akan kita gunakan adalah sebagai berikut:

In [ ]:
data = [['Agus','Fisika',100],['Windy','Fisika',200],
        ['Budi','Biologi',200],['Dina','Fisika',140],
        ['Bayu','Fisika',50],['Dedi','Biologi',50]]

kolom = ['nama','jurusan','nilai']

df = spark.createDataFrame(data,kolom)

df.show()

+-----+-------+-----+
| nama|jurusan|nilai|
+-----+-------+-----+
| Agus| Fisika|  100|
|Windy| Fisika|  200|
| Budi|Biologi|  200|
| Dina| Fisika|  140|
| Bayu| Fisika|   50|
| Dedi|Biologi|   50|
+-----+-------+-----+



Kita akan menambahkan satu kolom bernama 'level', berdasar kolom 'nilai', dengan aturan:
- Jika nilai kurang dari 100 maka level = ‘Low’.
- Jika nilai antara 100 sampai kurang dari 150 maka level = ‘Medium’.
- Jika nilai 150 ke atas maka level = ‘High’.


##withColumn dan when-otherwise pada DataFrame

Cara pertama adalah kita menggunakan fungsi `withColumn()` yang digabungkan dengan fungsi-fungsi `when()` dan `otherwise()`.

Sintaks withColumn dengan when-otherwise adalah :

`df.withColumn(nama_kolom, ekspresi-when-otherwise)`




In [ ]:
df.withColumn("level", F.when(df.nilai < 100,"Low")
                                 .when(df.nilai < 150,"Medium")
                                 .otherwise("High")).show()

+-----+-------+-----+------+
| nama|jurusan|nilai| level|
+-----+-------+-----+------+
| Agus| Fisika|  100|Medium|
|Windy| Fisika|  200|  High|
| Budi|Biologi|  200|  High|
| Dina| Fisika|  140|Medium|
| Bayu| Fisika|   50|   Low|
| Dedi|Biologi|   50|   Low|
+-----+-------+-----+------+



##select dan when-otherwise pada DataFrame

Selain menggunakan `withColumn()`, kita bisa juga menggunakan fungsi `select()` untuk mengembalikan kolom-kolom tertentu.

Sintaksnya adalah sebagai berikut

`df.select(list_kolom, ekspresi-when-otherwise.alias(nama_kolom_baru))`


In [15]:
df.select(F.when(df.nilai < 100,"Low")
    .when(df.nilai < 150,"Medium")
    .otherwise("High")).show()

+-----------------------------------------------------------------------------+
|CASE WHEN (nilai < 100) THEN Low WHEN (nilai < 150) THEN Medium ELSE High END|
+-----------------------------------------------------------------------------+
|                                                                       Medium|
|                                                                         High|
|                                                                         High|
|                                                                       Medium|
|                                                                          Low|
|                                                                          Low|
+-----------------------------------------------------------------------------+



Untuk menentukan nama kolom yang baru, kita menggunakan fungsi `spark.sql.Column.alias`. Untuk menyertakan kolom-kolom lain digunakan fungsi `spark.sql.functions.cols`

In [ ]:
df.select(F.col("*"), F.when(df.nilai < 100,"Low")
                    .when(df.nilai < 150,"Medium")
                    .otherwise("High").alias("level")).show()

+-----+-------+-----+-------+------+
| nama|jurusan|nilai|jurusan| level|
+-----+-------+-----+-------+------+
| Agus| Fisika|  100| Fisika|Medium|
|Windy| Fisika|  200| Fisika|  High|
| Budi|Biologi|  200|Biologi|  High|
| Dina| Fisika|  140| Fisika|Medium|
| Bayu| Fisika|   50| Fisika|   Low|
| Dedi|Biologi|   50|Biologi|   Low|
+-----+-------+-----+-------+------+



##Menambah beberapa kolom baru pada DataFrame dengan select() dan when-otherwise

Dengan menggunakan perintah `select`, kita bisa menggunakan beberapa ekspresi when-otherwise untuk membuat beberapa kolom baru sekaligus.

Sintaksnya adalah:
`df.select(list_kolom, ekspresi1, ekspresi2)`


In [ ]:
data2 = [['Agus','Fisika',100,180],['Windy','Fisika',200,100],
        ['Budi','Biologi',200,150],['Dina','Fisika',140,200],
        ['Bayu','Fisika',50,30],['Dedi','Biologi',50,180]]

kolom = ['nama','jurusan','nilai1','nilai2']

df2 = spark.createDataFrame(data2,kolom)

expr1 = F.when(df2.nilai1<150,"Low").otherwise("High").alias("level1")
expr2 = F.when(df2.nilai2<150,"Low").otherwise("High").alias("level2")

df2.select(F.col("*"),expr1,expr2).show()

+-----+-------+------+------+------+------+
| nama|jurusan|nilai1|nilai2|level1|level2|
+-----+-------+------+------+------+------+
| Agus| Fisika|   100|   180|   Low|  High|
|Windy| Fisika|   200|   100|  High|   Low|
| Budi|Biologi|   200|   150|  High|  High|
| Dina| Fisika|   140|   200|   Low|  High|
| Bayu| Fisika|    50|    30|   Low|   Low|
| Dedi|Biologi|    50|   180|   Low|  High|
+-----+-------+------+------+------+------+



Kita juga bisa memilih kolom mana saja yang akan kita sertakan dalam DataFrame yang baru

In [ ]:
df2.select(F.col("nama"),F.col("jurusan"),expr1,expr2).show()

+-----+-------+------+------+
| nama|jurusan|level1|level2|
+-----+-------+------+------+
| Agus| Fisika|   Low|  High|
|Windy| Fisika|  High|   Low|
| Budi|Biologi|  High|  High|
| Dina| Fisika|   Low|  High|
| Bayu| Fisika|   Low|   Low|
| Dedi|Biologi|   Low|  High|
+-----+-------+------+------+

